In [1]:
from common.np import *

# Chapter 4. word2vec 속도 개선

## \# 4.3 개선판 word2vec 학습

- 지금까지 기존 word2vec을 개선해본 내용:
    - Embedding 계층 사용
    - 네거티브 샘플링 기법 사용
    
    
- 이 개선안들을 신경망에 직접 적용시켜보자


- 이전에 구현했던 `SimpleCBOW` 클래스를 개선한 `CBOW`를 구현

In [2]:
from common.layer import Embedding, NegativeSamplingLoss

class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        
        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')
        
        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
        layers = self.in_layers + [self.ns_loss]
        
        # 모든 가중치와 기울기 모으기
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads
            
        # 단어의 분산표현 저장
        self.word_vecs = W_in
        
        
    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss
    
    
    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None

---
## \# 4.3.2 CBOW 모델 학습

- 이번 학습은 오래 걸리므로 GPU를 사용할 수 있는 `cupy` 라이브러리를 이용해서 학습한다.

#### cpu와 gpu를 원하는 대로 사용할 수 있도록 데이터타입을 변환해주는 함수

In [3]:
def to_cpu(x):
    import numpy
    if type(x) == numpy.ndarray:
        return x
    return np.asnumpy(x)

In [4]:
def to_gpu(x):
    import cupy
    if type(x) == cupy.ndarray:
        return x
    return cupy.asarray(x)

### \# CBOW 학습 코드

In [5]:
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from common.net import CBOW
from common.util import create_contexts_target, to_cpu, to_gpu
from common.dataset import PTB

In [6]:
from common import config
config.GPU = True

#### 1. 하이퍼파라미터 설정

In [7]:
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

#### 2. 데이터 가져오기

In [8]:
ptb = PTB()
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
if GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

Done


#### 3. 모델, trainer 생성

In [9]:
model = CBOW(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

#### 4. 학습 시작

In [11]:
trainer.fit(contexts, target, max_epoch, batch_size, log_step=4500)
trainer.plot()

time 33s | epoch 1 | step 4500 / 9295 | loss 4.19
time 67s | epoch 1 | step 9000 / 9295 | loss 4.12
time 102s | epoch 2 | step 4500 / 9295 | loss 4.18


KeyboardInterrupt: 

### 모델 저장
- `.pkl` 파일로 모델에 학습된 가중치를 저장

In [ ]:
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)

---
## \# 4.3.3 CBOW 모델 평가

- `most_similar` 함수로 가장 비슷한 단어를 잘 찾아내는지 평가

In [ ]:
import pickle
from common.util import most_similar
from common import config
config.GPU = True

In [ ]:
pkl_file = 'cbow_params.pkl'
with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

In [ ]:
queries = ['you', 'year', 'car', 'toyota']
for query in queries:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

- `analogy` 함수로 단어 간의 관계를 잘 학습했는지 평가

In [ ]:
from common.util import analogy

In [ ]:
analogy('man', 'king', 'woman', word_to_id, id_to_word, word_vecs, top=5)
analogy('take', 'took', 'go', word_to_id, id_to_word, word_vecs, top=5)
analogy('car', 'cars', 'child', word_to_id, id_to_word, word_vecs, top=5)
analogy('good', 'better', 'bad', word_to_id, id_to_word, word_vecs, top=5)

In [ ]:
analogy('bread', 'eat', 'beer', word_to_id, id_to_word, word_vecs, top=5)